Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (529114, 28, 28) (529114,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (529114, 784) (529114, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# Logistic Regression
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  regularizer = 5e-4 * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularizer
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 18.921
Minibatch accuracy: 4.7%
Validation accuracy: 15.9%
Minibatch loss at step 500 : 2.10521
Minibatch accuracy: 76.6%
Validation accuracy: 76.9%
Minibatch loss at step 1000 : 1.71902
Minibatch accuracy: 77.3%
Validation accuracy: 78.2%
Minibatch loss at step 1500 : 1.14345
Minibatch accuracy: 82.8%
Validation accuracy: 79.5%
Minibatch loss at step 2000 : 1.02007
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 2500 : 0.903959
Minibatch accuracy: 82.0%
Validation accuracy: 81.6%
Minibatch loss at step 3000 : 1.06568
Minibatch accuracy: 79.7%
Validation accuracy: 81.9%
Test accuracy: 88.0%


In [7]:
# Neural Network
batch_size = 128
n_hidden = 1024

def model(X, weights, biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(X, weights['h1']), biases['b1']))
    return tf.matmul(layer_1, weights['out']) + biases['out']
    

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
        'h1':  tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])), 
        'out': tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
    }
  biases = {
        'b1':  tf.Variable(tf.zeros([n_hidden])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
  
  # Training computation.
  pred = model(tf_train_dataset, weights, biases)
  regularizer = 5e-4 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(biases['b1']))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(pred, tf_train_labels)) + regularizer
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights, biases))
  test_prediction  = tf.nn.softmax(model(tf_test_dataset, weights,  biases))

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 454.375
Minibatch accuracy: 10.9%
Validation accuracy: 25.9%
Minibatch loss at step 500 : 125.168
Minibatch accuracy: 85.9%
Validation accuracy: 80.9%
Minibatch loss at step 1000 : 97.0876
Minibatch accuracy: 77.3%
Validation accuracy: 83.0%
Minibatch loss at step 1500 : 73.7226
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Minibatch loss at step 2000 : 57.2774
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 2500 : 44.7717
Minibatch accuracy: 80.5%
Validation accuracy: 84.7%
Minibatch loss at step 3000 : 35.2392
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Test accuracy: 91.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 512 

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 457.497
Minibatch accuracy: 13.3%
Validation accuracy: 33.3%
Minibatch loss at step 100 : 169.759
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Minibatch loss at step 200 : 177.844
Minibatch accuracy: 78.1%
Validation accuracy: 80.2%
Minibatch loss at step 300 : 150.56
Minibatch accuracy: 73.4%
Validation accuracy: 79.7%
Minibatch loss at step 400 : 137.559
Minibatch accuracy: 82.0%
Validation accuracy: 79.0%
Minibatch loss at step 500 : 123.085
Minibatch accuracy: 86.7%
Validation accuracy: 81.4%
Test accuracy: 86.4%


The test accuracy has dropped from 91.3% to 87.0%

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
# Neural Network
batch_size = 128
n_hidden = 1024

def model(X, weights, biases, train=False):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(X, weights['h1']), biases['b1']))
    if train:
        layer_1 = tf.nn.dropout(layer_1, 0.5, seed=100)
    return tf.matmul(layer_1, weights['out']) + biases['out']
    

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
        'h1':  tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden])), 
        'out': tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
    }
  biases = {
        'b1':  tf.Variable(tf.zeros([n_hidden])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
  
  # Training computation.
  pred = model(tf_train_dataset, weights, biases, train=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(pred, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights, biases, train=False))
  test_prediction  = tf.nn.softmax(model(tf_test_dataset, weights,  biases, train=False))

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 491.65
Minibatch accuracy: 9.4%
Validation accuracy: 29.8%
Minibatch loss at step 500 : 9.24252
Minibatch accuracy: 74.2%
Validation accuracy: 79.6%
Minibatch loss at step 1000 : 13.7971
Minibatch accuracy: 73.4%
Validation accuracy: 80.7%
Minibatch loss at step 1500 : 5.77647
Minibatch accuracy: 71.1%
Validation accuracy: 81.1%
Minibatch loss at step 2000 : 11.2837
Minibatch accuracy: 70.3%
Validation accuracy: 80.0%
Minibatch loss at step 2500 : 11.4581
Minibatch accuracy: 66.4%
Validation accuracy: 78.7%
Minibatch loss at step 3000 : 3.95935
Minibatch accuracy: 75.8%
Validation accuracy: 81.2%
Test accuracy: 87.3%


In [12]:
# Extreme Overfitting Case
num_steps = 512 

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 516.566
Minibatch accuracy: 8.6%
Validation accuracy: 27.0%
Minibatch loss at step 100 : 46.5599
Minibatch accuracy: 75.8%
Validation accuracy: 78.7%
Minibatch loss at step 200 : 69.811
Minibatch accuracy: 70.3%
Validation accuracy: 81.7%
Minibatch loss at step 300 : 46.6203
Minibatch accuracy: 78.1%
Validation accuracy: 81.0%
Minibatch loss at step 400 : 24.1695
Minibatch accuracy: 77.3%
Validation accuracy: 80.0%
Minibatch loss at step 500 : 15.285
Minibatch accuracy: 75.8%
Validation accuracy: 81.6%
Test accuracy: 85.1%


With fewer steps, we got higher Test Accuracy when dropout was introduced.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [17]:
# Neural Network
batch_size = 128
n_hidden1 = 400
n_hidden2 = 50

def model(X, weights, biases, dropout=False):
    if dropout:
        layer_1 = tf.nn.relu(tf.add(tf.matmul(X, weights['h1']), biases['b1']))
        layer_1 = tf.nn.dropout(layer_1, 0.5, seed=100)
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
        layer_2 = tf.nn.dropout(layer_2, 0.5, seed=100)
    else:
        layer_1 = tf.nn.relu(tf.add(tf.matmul(X, weights['h1']), biases['b1']))
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    return tf.matmul(layer_2, weights['out']) + biases['out']
    

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {
        'h1':  tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden1])), 
        'h2':  tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2])),
        'out': tf.Variable(tf.truncated_normal([n_hidden2, num_labels]))
    }
  biases = {
        'b1':  tf.Variable(tf.zeros([n_hidden1])),
        'b2':  tf.Variable(tf.zeros([n_hidden2])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
  
  # Training computation.
  pred = model(tf_train_dataset, weights, biases, dropout=False)
  regularizer = 5e-4 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(biases['b1']))
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, tf_train_labels)) + regularizer
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss) # Adam Optimizer
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(pred)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights, biases, dropout=False))
  test_prediction  = tf.nn.softmax(model(tf_test_dataset, weights,  biases, dropout=False))

In [18]:
num_steps = 40001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print "\nMinibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
      print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized

Minibatch loss at step 0 : 1026.29
Minibatch accuracy: 8.6%
Validation accuracy: 8.8%
Test accuracy: 8.4%

Minibatch loss at step 1000 : 66.6131
Minibatch accuracy: 46.1%
Validation accuracy: 46.1%
Test accuracy: 50.7%

Minibatch loss at step 2000 : 58.793
Minibatch accuracy: 39.1%
Validation accuracy: 42.5%
Test accuracy: 47.9%

Minibatch loss at step 3000 : 55.402
Minibatch accuracy: 50.8%
Validation accuracy: 49.0%
Test accuracy: 54.9%

Minibatch loss at step 4000 : 50.1738
Minibatch accuracy: 58.6%
Validation accuracy: 54.0%
Test accuracy: 59.9%

Minibatch loss at step 5000 : 43.3879
Minibatch accuracy: 58.6%
Validation accuracy: 56.6%
Test accuracy: 62.7%

Minibatch loss at step 6000 : 34.8744
Minibatch accuracy: 64.8%
Validation accuracy: 63.4%
Test accuracy: 70.3%

Minibatch loss at step 7000 : 25.9397
Minibatch accuracy: 72.7%
Validation accuracy: 69.9%
Test accuracy: 77.3%

Minibatch loss at step 8000 : 18.2114
Minibatch accuracy: 72.7%
Validation accuracy: 72.2%
